# `site-analysis` example

In [1]:
from pymatgen.io.vasp import Poscar, Xdatcar
import numpy as np
import operator
from site_analysis import PolyhedralSite, Atom, Analysis, get_vertex_indices, AtomsTrajectory, SitesTrajectory

The first step is to identify the atoms that define the coordination polyhedra vertices.  
To do this we load a `POSCAR` file where every octahedral site is occupied by a Na atom.

In [2]:
all_na_structure = Poscar.from_file('na_sn_all_na_ext.POSCAR.vasp').structure
vertex_species = 'S'
centre_species = 'Na'

We then use the `get_vertex_indices()` function to find the six closest S to each Na (within a cutoff of 4.3 Å).  
This returns a nested list, where each sublist contains the S indices for a single polyedron.  
Note: this index counts from 1, and ignores other species in the structure (so is not affected by species order).

In [3]:
# find atom indices (within species) for all polyhedra vertex atoms
vertex_indices = get_vertex_indices(all_na_structure, centre_species=centre_species, 
                                    vertex_species=vertex_species, cutoff=4.3)
print(vertex_indices[:4])

[[27, 29, 59, 61, 83, 85], [19, 21, 51, 53, 91, 93], [9, 15, 41, 47, 65, 71], [7, 23, 33, 49, 73, 90]]


We can now use these vertex ids to define our `Polyhedron` objects.  
We also define our `Atom` objects, using an example structure with the correct Na stoichiometry.
These lists of polyhedra and atoms are then used to create an `Analysis` object.

In [4]:
structure = Poscar.from_file('POSCAR').structure
# create Polyhedron objects
sites = [PolyhedralSite(vertex_species=vertex_species, vertex_indices=vi) for vi in vertex_indices]
# create Atom objects
atoms = [Atom(species_string=centre_species) for site in structure if site.species_string is 'Na']
analysis = Analysis(sites, atoms)

The `Analysis` object provides the main interface for working with the `Polyhedron` and `Atom` objects.  
e.g. to get a summary of the polyhedron coordination numbers:

In [5]:
analysis.site_coordination_numbers()

Counter({6: 136})

To analyse the site occupation for a particular `pymatgen` `Structure`:

In [6]:
analysis.analyse_structure(structure)

The list of sites occupied by each atom can now be accessed using `analysis.atom_sites`

In [7]:
np.array(analysis.atom_sites)

array([ 10,  93, 100,  15,  17,  69,  67,  24,  97,  84,  59,  57,  87,
        55,  65,  16,  66,  13,  92,  14,  95,  51,  61,  86,  54,  25,
        79,  83,  33,  35,   6,  41,   5,  43,   4,  36,  34,  42,   8,
        40,  12,  94,  50,  68,  27,  82,   7,  63,  70,  49,   1,  98,
        23,  91,  62,  56,   9,  31, 102,  73,  32,   2,  72,  99,   3,
        90,  22,  71,  88,  26,  85,  21,  47,  37,  81,  18,  52,  64,
        89,  45,  60,  53,  58,  44,  46, 101,  19,  80])

The list of atoms occupying each site can be accessed using `analysis.site_occupations`.  
The occupations of each site are stored as a list of lists, as each site can have zero, one, or multiple atoms occupying it.

In [8]:
print(analysis.site_occupations)

[[51], [62], [65], [35], [33], [31], [47], [39], [57], [1], [], [41], [18], [20], [4], [16], [5], [76], [87], [], [72], [67], [53], [8], [26], [70], [45], [], [], [], [58], [61], [29], [37], [30], [36], [74], [], [], [40], [32], [38], [34], [84], [80], [85], [73], [], [50], [43], [22], [77], [82], [25], [14], [56], [12], [83], [11], [81], [23], [55], [48], [78], [15], [17], [7], [44], [6], [49], [68], [63], [60], [], [], [], [], [], [27], [88], [75], [46], [28], [10], [71], [24], [13], [69], [79], [66], [54], [19], [2], [42], [21], [], [9], [52], [64], [3], [86], [59], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]


A *trajectory* consists of a series of site occupations over multiple timesteps. A single timestep can be processed using the `analysis.append_timestep()` method:

In [9]:
analysis.append_timestep(structure)

There are two ways to think about a site-projected trajectory:  
1. From an atom-centric perspective. Each atom visits a series of sites, and occupies one site each timestep.
2. From a site-centric perspective. Each site is visited by a series of atoms, and has zero, one, or more atoms occupying it at each timestep.

These two trajectory types can be accessed using the `Atom.trajectory` and `Site.trajectory` attributes:

In [10]:
print(atoms[3].trajectory)

[15]


In [11]:
print(sites[3].trajectory)

[[35]]


The combined atom trajectory can be accessed using `analysis.atoms_trajectory`.

In [12]:
print(analysis.atoms_trajectory)

[[10, 93, 100, 15, 17, 69, 67, 24, 97, 84, 59, 57, 87, 55, 65, 16, 66, 13, 92, 14, 95, 51, 61, 86, 54, 25, 79, 83, 33, 35, 6, 41, 5, 43, 4, 36, 34, 42, 8, 40, 12, 94, 50, 68, 27, 82, 7, 63, 70, 49, 1, 98, 23, 91, 62, 56, 9, 31, 102, 73, 32, 2, 72, 99, 3, 90, 22, 71, 88, 26, 85, 21, 47, 37, 81, 18, 52, 64, 89, 45, 60, 53, 58, 44, 46, 101, 19, 80]]


and the combined site trajectory can be accessed using `analysis.sites_trajectory`.

In [13]:
print(analysis.sites_trajectory)

[[[51], [62], [65], [35], [33], [31], [47], [39], [57], [1], [], [41], [18], [20], [4], [16], [5], [76], [87], [], [72], [67], [53], [8], [26], [70], [45], [], [], [], [58], [61], [29], [37], [30], [36], [74], [], [], [40], [32], [38], [34], [84], [80], [85], [73], [], [50], [43], [22], [77], [82], [25], [14], [56], [12], [83], [11], [81], [23], [55], [48], [78], [15], [17], [7], [44], [6], [49], [68], [63], [60], [], [], [], [], [], [27], [88], [75], [46], [28], [10], [71], [24], [13], [69], [79], [66], [54], [19], [2], [42], [21], [], [9], [52], [64], [3], [86], [59], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]]


Example of processing a simulation trajectory using the `XDATCAR` file:  
(using `analysis.reset()` to reset the trajectory data.)

In [14]:
analysis.reset()

xdatcar = Xdatcar('XDATCAR')

for timestep, s in enumerate(xdatcar.structures):
    analysis.append_timestep(s, t=timestep)

Checking which sites has Na(4) visited:  
(note use of `analysis.at` as shorthand for `analysis.atoms_trajectory`)

In [15]:
print(analysis.atom_by_index(4).trajectory) # convert to a numpy array to then use numpy array slicing to extract a single atom trajectory.

[15, 15, 15, 15, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73, 73]


Na(4) starts in site 15, and moves to site 73 at timestep 5.  
The same information can be seen by querying the site occupation data for sites 15 and 73:  
(note use of `analysis.st` as shorthand for `analysis.sites_trajectory`)

In [16]:
print(analysis.site_by_index(15).trajectory)
print(analysis.site_by_index(73).trajectory)

[[4], [4], [4], [4], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]
[[], [], [], [], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4], [4]]
